In [1]:
from perseuspy import pd

In [2]:
import opendvp

In [3]:
adata = opendvp.io.import_perseus(
    path_to_perseus_txt="test_data/io/Perseus_v1.6.15.0.txt",
    n_var_metadata_rows=5
)

10:58:40.67 | INFO | Reading Perseus file from: test_data/io/Perseus_v1.6.15.0.txt
10:58:40.69 | INFO | Perseus DataFrame shape: (3526, 16)
10:58:40.70 | INFO | Data matrix shape: (11, 3526)
10:58:40.71 | SUCCESS | AnnData object created from Perseus file.


/opt/homebrew/Caskroom/mambaforge/base/envs/opendvp-sdata/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/homebrew/Caskroom/mambaforge/base/envs/opendvp-sdata/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [4]:
adata.shape

(11, 3526)

In [5]:
adata.obs.columns.tolist()

['Column Name',
 'Heart_Condition',
 'Ischemia_region',
 'Sample_type',
 'Replicate']